In [31]:
import torch
from torch import nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import pandas as pd
from tqdm import trange
import time

In [32]:
if torch.cuda.is_available(): device = 'cuda'
elif torch.backends.mps.is_available(): device = 'mps'
else: device = 'cpu'
print(f'Using device: {device}')

torch.manual_seed(42);

Using device: mps


In [33]:
# load names data
words = open("data/names.txt", "r").read().splitlines()
words_len = len(words)
max_len = max(len(w) for w in words)

print(f'number of words: {words_len}')
print(f'longest word: {max_len}')
print(f'sample words: {words[:8]}')

number of words: 32033
longest word: 15
sample words: ['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [34]:
# setup chars, vocab_size and
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0

itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(f'vocab size: {vocab_size}')

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
vocab size: 27


In [35]:
# is this necessary?

import random
random.seed(42)
random.shuffle(words)

In [36]:
ctx_len = max_len

def build_dataset(words: str):
    X, Y = [], []
    for w in words:
        context = [0] * ctx_len
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X).to(device)
    Y = torch.tensor(Y).to(device)
    print(X.shape, Y.shape)
    return X, Y

n = int(0.9*len(words))
Xtr, Ytr = build_dataset(words[:n])
Xval, Yval = build_dataset(words[n:])

torch.Size([205280, 15]) torch.Size([205280])
torch.Size([22866, 15]) torch.Size([22866])


In [37]:
print(Ytr[:25])
print([itos[ch.item()] for ch in Ytr[:25]])

tensor([25, 21,  8,  5, 14,  7,  0,  4,  9, 15, 14,  4, 18,  5,  0, 24,  1, 22,
         9,  5, 14,  0, 10, 15, 18], device='mps:0')
['y', 'u', 'h', 'e', 'n', 'g', '.', 'd', 'i', 'o', 'n', 'd', 'r', 'e', '.', 'x', 'a', 'v', 'i', 'e', 'n', '.', 'j', 'o', 'r']


In [38]:
# predict the next letter given previous n number of letters where n <= 8
Xtr[5], Ytr[5], Xtr[6]

(tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 25, 21,  8,  5, 14],
        device='mps:0'),
 tensor(7, device='mps:0'),
 tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0, 25, 21,  8,  5, 14,  7],
        device='mps:0'))

In [48]:
class Linear:
    def __init__(self, fet_in: int, fet_out: int, bias: bool=True, device=None):
        self.weight = torch.randn((fet_in, fet_out), device=device) / fet_in**0.5 # kaiming init
        self.bias = torch.zeros(fet_out, device=device) if bias else None
    
    def __call__(self, x: torch.Tensor):
        self.out = x @ self.weight
        if self.bias is not None: self.out += self.bias
        return self.out
    
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])

class ReLU:
    def __call__(self, x: torch.Tensor):
        self.out = F.relu(x)
        return self.out
    
    def parameters(self):
        return []

class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1, device=None):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        # params
        self.gamma = torch.ones(dim, device=device)
        self.beta = torch.zeros(dim, device=device)
        # buffs
        self.running_mean = torch.zeros(dim, device=device)
        self.running_var = torch.ones(dim, device=device)

    def __call__(self, x):
        if self.training:
            xmean = x.mean(0, keepdim=True)
            xvar = x.var(0, keepdim=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        # update buffs
        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]
  
class Sequential:
    def __init__(self, layers: list):
        self.layers = layers
    
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        self.out = x
        return self.out
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [53]:
# starting loss should be -ln(1/vocab_size)
from numpy import log as nllog
-nllog(1/vocab_size)

np.float64(3.295836866004329)

In [54]:
n_hidden = 56

model = Sequential([
    Linear(ctx_len, n_hidden, device=device), BatchNorm1d(n_hidden, device=device), ReLU(),
    Linear(n_hidden, n_hidden, device=device), BatchNorm1d(n_hidden, device=device), ReLU(),
    Linear(n_hidden, n_hidden, device=device), BatchNorm1d(n_hidden, device=device), ReLU(),
    Linear(n_hidden, n_hidden, device=device), BatchNorm1d(n_hidden, device=device), ReLU(),
    Linear(n_hidden, vocab_size, device=device),
])

with torch.no_grad():
    model.layers[-1].weight *= 0.1

parameters = model.parameters()
print(f'num of params in model: {sum(p.nelement() for p in parameters)}')
for p in parameters:
    p.requires_grad = True

num of params in model: 12459


In [55]:
epochs = 50000
eval_iter = 10000
batch_size = 32
start_time = time.time()

for i in trange(epochs):
    # mini batch
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    Xb, Yb = Xtr[ix].float(), Ytr[ix]
    
    # forward pass
    logits = model(Xb)
    loss = F.cross_entropy(logits, Yb)
    
    # compute gradients
    for p in parameters:
        p.grad = None
    loss.backward()
  
    # update
    lr = 0.1 if i < 20000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
  
    # stats
    if i % eval_iter == 0:
        print(f'{i}: {loss.item():4f}')

end_time = time.time()
print(f'time: {end_time - start_time}:4f')

  0%|                                        | 14/50000 [00:00<22:38, 36.80it/s]

0: 3.282554


 20%|███████▏                            | 10024/50000 [01:19<05:23, 123.66it/s]

10000: 2.447887


 40%|██████████████▍                     | 20019/50000 [02:41<04:14, 117.78it/s]

...

30000: 2.237483


 80%|████████████████████████████▊       | 40020/50000 [05:27<01:20, 123.75it/s]

40000: 2.303773


100%|████████████████████████████████████| 50000/50000 [06:51<00:00, 121.49it/s]

time: 411.56100487709045:4f


In [56]:
@torch.no_grad()
def split_loss(split: str):
    x,y = {
        'train': (Xtr, Ytr),
        'val': (Xval, Yval),
    }[split]
    logits = model(x.float())
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())
  
split_loss('train')
split_loss('val')

train 2.362396478652954
val 2.3741557598114014


In [62]:
for _ in range(20):
    out = []
    context = [0] * ctx_len
    while True:
        logits = model(torch.tensor([context], dtype=torch.float, device=device))
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))

glrvmj.
tezacrznktud.
nicrxz.
xfsqzwmaftkv.
pa.
dnuekwa.
jphyltdrh.
znuvczhafwmvtiwwsotibaziauelmurcvruejqgshbahjlak.
apnfwio.
nkafbtmilyaa.
yut.
celqbag.
asjepu.
sv.
ypdxajh.
gohiekqyhqhnmgvbaozakabgnhr.
o.
ejbpyhlairinah.
mjjlmsoapfli.
xuxqeojldcszho.
